In [25]:
import logging
from typing import Any, Dict
import duckdb
import pandas as pd
import sqlglot
import yaml
from pathlib import Path


class DataUnitTest:

    def __init__(self, path: str, model_name: str, model_sql: str):
        self.path = path
        self.model_name = model_name
        self.model_sql = model_sql
        self.df = self.fetch_test_dataset(self.path)

    def fetch_test_dataset(self, path) -> pd.DataFrame:
        try:
            return pd.read_csv(path)
        except Exception as e:
            raise Exception(f"Error fetching test dataset: {e}")

    def extract_ctes_from_sql(self):
        parsed = sqlglot.parse(self.model_sql)
        statements = {}
        for stmt in parsed:
            try:
                if stmt.ctes:
                    cte_map = {}
                    statements['CTE'] = str(stmt)
                    for idx, cte in enumerate(stmt.ctes):
        
                        cte_str = str(cte)
                        name = cte_str.split('AS')[0].strip()
                        query = cte_str.split('AS', 1)[1].strip()
                        query = sqlglot.transpile(query, write="duckdb", identify=True, pretty=True)[0]
                        cte_map[cte.alias] = query
            except:
                pass

            if type(stmt).__name__ == 'Command':
                stmt_type = type(stmt).expression
                statements[str(stmt_type)] = stmt
            else:
                stmt_type = type(stmt).__name__
                statements[str(stmt_type)] = stmt 
        return cte_map, statements

    def fetch_cte_data(self, cte_map: Dict[str, str]) -> Dict[str, Any]:
        sql_df = self.df.copy()
        cte_data = {}
        for cte_name, cte_query in cte_map.items():
            try:
                cte_query = cte_query.removesuffix(')').removeprefix('(').strip()
                cte_query = cte_query.replace("loans", "sql_df")
                cte_df = duckdb.query(cte_query).fetch_arrow_table().to_pandas()
                cte_data[cte_name] = cte_df.to_dict('records')
            except Exception as e:
                logging.error(f"Error fetching CTE data for {cte_name}: {e}")
                raise
        return cte_data

    def generate_test_yaml(self) -> str:
        ctes, statements = self.extract_ctes_from_sql()
        test_structure = {
            TEST_ID: {
                "model": self.model_name,
                "inputs": {
                    INPUT_MODEL: {
                        "rows": self.df.to_dict('records')
                    }
                },
                "outputs": {
                    "query": {},
                    "ctes": {}
                }
            }
        }
        cte_data = self.fetch_cte_data(ctes)
        for cte_name, data in cte_data.items():
            test_structure[TEST_ID]["outputs"]["ctes"][cte_name] = {"rows": data}
    
        try:
            loans = self.df.copy()
            main_query_output = duckdb.query(str(statements['CTE'])).fetch_arrow_table().to_pandas()
            test_structure[TEST_ID]["outputs"]["query"]["rows"] = main_query_output.to_dict('records')
        except Exception as e:
            logging.error(f"Error in main query: {e}")
            raise
    
        return yaml.dump(test_structure)

    def run(self) -> str:
        return self.generate_test_yaml()


def modify_yaml(file_path, data):
    data = yaml.load(data, Loader=yaml.FullLoader)
    model_value = data.get('model', None)
    if TEST_ID in data and 'inputs' in data[TEST_ID]:
        data[TEST_ID] = {
            'model': model_value,
            **data[TEST_ID]
        }
    with open(file_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False, sort_keys=False)


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    
    root = Path().cwd().parent
    METRIC_DS_NM = "metric_loans"
    DATA_FILE = root / f"data/test_{METRIC_DS_NM}.csv"
    OUTPUT_FILE = root / f"tests/test_{METRIC_DS_NM}.yaml"
    SQL_FILE = root / f"sql/test_{METRIC_DS_NM}.sql"
    with open(SQL_FILE, 'r') as file:
        MODEL_SQL = file.read()

    TEST_ID = f"sqlmesh_test_suite_id"
    INPUT_MODEL = f"sqlmesh_test_id.{METRIC_DS_NM}_input_model"
    MODEL_NM = f"sqlmesh_test_id.{METRIC_DS_NM}_full_model"

    tester = DataUnitTest(DATA_FILE, MODEL_NM, MODEL_SQL)
    yaml_output = tester.run()
    modify_yaml(OUTPUT_FILE, yaml_output)

FileNotFoundError: [Errno 2] No such file or directory: 'sql/test_metric_loans.sql'

In [4]:
import pandas as pd
from faker import Faker

# Initialize faker
fake = Faker()

# Number of rows
num_rows = 1000

# Generate the data
data = {
    'loan_id': range(1, num_rows + 1),
    'applicant_name': [fake.name() for _ in range(num_rows)],
    'applicant_age': [fake.random_int(min=20, max=60) for _ in range(num_rows)],
    'loan_amount': [fake.random_number(digits=6) for _ in range(num_rows)],
    'loan_duration': [fake.random_int(min=12, max=60) for _ in range(num_rows)],  # 12 to 60 months
    'interest_rate': [fake.pyfloat(left_digits=2, right_digits=2, positive=True, min_value=2.0, max_value=15.0) for _ in range(num_rows)],  # 2% to 15%
    'application_date': [fake.date_this_decade() for _ in range(num_rows)],
    'loan_type': [fake.random_element(elements=('Home', 'Auto', 'Personal')) for _ in range(num_rows)]
}

# Convert to DataFrame
df_loans = pd.DataFrame(data)

# Display a sample of the data
print(df_loans.head())
df_loans.to_csv('loans_dataset.csv', index=False)


   loan_id     applicant_name  applicant_age  loan_amount  loan_duration  \
0        1        Taylor Ball             39       924966             52   
1        2   Vanessa Anderson             35       908595             54   
2        3  Christopher White             25       947483             49   
3        4       Kayla Golden             32       670765             52   
4        5  Haley Fitzpatrick             37       841284             19   

   interest_rate application_date loan_type  
0           8.45       2022-04-02      Auto  
1          11.75       2023-06-17  Personal  
2           8.19       2022-08-07      Auto  
3           7.65       2021-03-04      Auto  
4           3.94       2020-12-12      Auto  
